In [90]:
from __future__ import print_function

import os
import json
#import boto3
from datetime import datetime, timedelta
from urllib2 import urlopen
#from botocore.exceptions import ClientError
#from boto3.dynamodb.conditions import Key, Attr


In [74]:
SITE = 'https://bittrex.com/api/v1.1/public/getmarketsummaries'  # URL of the site to check, stored in the site environment variable, e.g. https://aws.amazon.com
#EXPECTED = os.environ['expected']  # String expected to be on the page, stored in the expected environment variable, e.g. Amazon
MARKETNAME = 'MarketName'
TIMESTAMP = 'TimeStamp'
#TRADINGSNS = os.environ['tradingSNS']
INDIVIDUALSUMMARYPREFIX = 'https://bittrex.com/Api/v2.0/pub/market/GetTicks?marketName='
INDIVIDUALSUMMARYPOSTFIX = '&tickInterval=oneMin&_=1499127220008'
MARKETLIMIT = 2
HOURINTEREST = int(336) #hours


In [5]:
#holdingStatusTable = dynamodb.Table('MarketHoldingStatus')
#here?

1

In [94]:
def retrieveMarketHistoricalData(rawMarketData):
    marketHistoricalData = dict()
    print('Total number of market is {}'.format(str(len(rawMarketData['result']))))
    listOfMarket = getListOfMarket(rawMarketData)
    
    timeStop = str(datetime.utcnow() - timedelta(hours = HOURINTEREST)).replace(' ', 'T')
    
    marketLimit = int(MARKETLIMIT)
    for market in listOfMarket:
        print(market)
        individualMarketUrl = INDIVIDUALSUMMARYPREFIX + market + INDIVIDUALSUMMARYPOSTFIX
        unfilledData = json.loads(urlopen(individualMarketUrl).read())['result']

        try:
            it = next(i for i in xrange(len(unfilledData)) if unfilledData[i]['T'] >= timeStop)
        except:
            print(market + ": no valid data within last " + str(HOURINTEREST) + " hours")
        else:
            cutUnfilledData = unfilledData[it:]
            marketHistoricalData[market] = cutUnfilledData
        finally:
            # Need to break before running limit
            marketLimit = marketLimit - 1
            if (marketLimit <= 0):
                break
    
    return marketHistoricalData
def getListOfMarket(rawMarketData):
    listOfMarket = list()
    for record in rawMarketData['result']:
        tradingPair = record[MARKETNAME]
        if (tradingPair.startswith('BTC')):
            listOfMarket.append(record[MARKETNAME])
    
    return listOfMarket

In [95]:
marketUrl = SITE
rawMarketData = json.loads(urlopen(marketUrl).read())
marketHistoricalData = retrieveMarketHistoricalData(rawMarketData)

Total number of market is 263
BTC-1ST
BTC-2GIVE


In [98]:
#loop every 5 min, get 5min data 
#get a list of oldest to newest timestamp, 5min apart, loop the list, for each timestamp, get the 5min before current-timestamp data

#ans=writeSignalHistory(tradingPair=pair,data=marketHistoricalData[pair],rwLength=60)
#rwLength = how far to look back, in min, same as HOURINTEREST
import datetime
import time
import calendar
pair='BTC-2GIVE'
data=marketHistoricalData[pair]
    #sort data to make sure its time ascending
data.sort(key=lambda x:x['T'])
startTS=calendar.timegm(datetime.datetime.strptime(data[0]['T'],"%Y-%m-%dT%H:%M:%S").timetuple())
endTS=calendar.timegm(datetime.datetime.strptime(data[-1]['T'],"%Y-%m-%dT%H:%M:%S").timetuple())
startTime=data[0]['T']
endTime=data[-1]['T']

rwLengthMin=60 #min 
a=range(startTS,endTS, 5*60 )


In [114]:
for i in range(len(a)):
    TS_i=range(startTS,endTS, 5*60)[i]
    DateTime_i=datetime.datetime.utcfromtimestamp(int(TS_i)).strftime('%Y-%m-%dT%H:%M:%S')
    #get data at ith DateTime 
    it = next(j for j in xrange(len(data)) if data[j]['T'] >= DateTime_i)
    data = data[:(it-1)]
    latestTS=calendar.timegm(datetime.datetime.strptime(data[-1]['T'],"%Y-%m-%dT%H:%M:%S").timetuple())
    print([TS_i, DateTime_i, latestTS])
    #here debug, how not to stop iteration 
    #next, update peakprice, etc. 
    #next, how to do the time loop in the outmost loop 
    #how to save the signal 

StopIteration: 

In [ ]:
#1. Read 2wks data for all pairs
#2. Input: 2wks data, all parameters; output: SignalHistory(dictionary, key=market, value=[timestamp, buy_indicator, sell_indicator])
    #First (outmost) loop: loop around two weeks datetime (two weeks before current UTC time), for every 5 min
        #For the current 5min window, get the end dateTime of the window 
        #Second loop: loop around all trading pairs, for each trading pair 
            #get the data corresponding to current window (compare the end dateTime of the current window and the data, find the lastest data before end of current window)
            #Update peak price (setHoldingStatus)
            #apply the trading logic and save the signalHistory 
#3. Merge the signlaHistory with the two weeks full data, output a dictionary (key=market,value=[timestamp, C, buy_indicator, sell_indicator] )
#4. Calculate the returns based on #3, plot the return curve etc. 

#(Mengye needs help with 1 and 3. She can do 3 and 4 by herself)

